# Investigation into drag force with varying velocity and constant fluid density and shape for coffee filter

An investigation was conducted by dropping a varying number of coffee filters under fixed experimental conditions. The terminal velocity was determined by using a motion sensor. Experimental results were collected as follows:

In [29]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from scipy.optimize import curve_fit

In [27]:
m = 0.88e-3 # kg 
g_constant = 9.81

data = [
    [0, 0],
    [0.940, m * g_constant],
    [1.34, 2 * m * g_constant],
    [1.7, 3 * m * g_constant],
    [2.1, 4 * m * g_constant],
]

In [28]:
df = pd.DataFrame(data, columns=["Terminal velocity (m/s)", "Force (kg‧ms-2)"])
df

,Terminal velocity (m/s),Force (kg‧ms-2)
0,0.00,0.000000
1,0.94,0.008633
2,1.34,0.017266
3,1.70,0.025898
4,2.10,0.034531


In [52]:
v = df["Terminal velocity (m/s)"]
f = df["Force (kg‧ms-2)"]

In [59]:
def quadratic_model(x, a, b, c):
    return a ** x *2 + b * x + c

def linear_model(x, a, b):
    return a * x + b

In [64]:
param, param_cov = curve_fit(linear_model, v ** 2, f)
param_q, param_cov_q = curve_fit(quadratic_model, v, f)

In [66]:
samples = 160
v_generated = np.linspace(v[0] ** 2, v[4] ** 2, num=samples)
f_best_fit = linear_model(v_generated, param[0], param[1])

v_generated_quadratic = np.linspace(v[0], v[4], num=samples)
f_best_fit_quadratic = quadratic_model(v_generated_quadratic, 
                                       param_q[0], 
                                       param_q[1], 
                                       param_q[2])

In [68]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=v_generated_quadratic, 
                          y=f_best_fit_quadratic, 
                          mode="lines", 
                          name="Best fit"))
fig1.add_trace(go.Scatter(x=v, y=f, mode="markers", name="Data values"))
fig1.update_layout(title="Drag force given varying terminal velocity")
fig1.update_layout(
    xaxis_title=r"$v \, (\frac{m}{s})$",
    yaxis_title=r"$F \, (\frac{\mathrm{kg} \cdot m}{s})$"
)
fig1.show()

In [69]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=v_generated, y=f_best_fit, mode="lines", name="Best fit"))
fig2.add_trace(go.Scatter(x=v ** 2, y=f, mode="markers", name="Data values"))
fig2.update_layout(title="Drag force given varying terminal velocity (linearized)")
fig2.update_layout(
    xaxis_title=r"$v^2 \, (\frac{m}{s})^2$",
    yaxis_title=r"$F \, (\frac{\mathrm{kg} \cdot m}{s})$"
)
fig2.show()

## Data Analysis

$$
k = \frac{F_2 - F_1}{v^2_2 - v^2_1}
$$

In [83]:
k = (f_best_fit[-1] - f_best_fit[0]) / ((v_generated[-1]) ** 2 - v_generated[0] ** 2)
print(f"{k} kg / m")

0.0017827052039678898 kg / m


We may now have the general form of the drag force equation:

$$
F_D = k v^2 + C
$$

The value of $k$ can reasonably be assumed to depend on the fluid density and shape (surface area), where:

$$
\rho = 1.25 \frac{\mathrm{kg}}{\mathrm{m}^3}
$$

$$
A = 0.0226 \mathrm{\ m^2}
$$

Thus, the drag force equation can be found to be:

$$
F_D = \kappa A \rho v^2
$$

where $\kappa = \frac{1}{2} C_D$ and $C_D$ is the drag coefficient.